In [ ]:
import urllib
import re
import random
import string
import operator
from collections import Counter,defaultdict




In [ ]:
'''
def text_filter(text: str) -> str:
    """
    文本过滤器：过滤掉文本数据中的标点符号和其他特殊字符
    :param text: 待过滤的文本
    :return: 过滤后的文本
    """
    # 使用正则表达式去除标点符号和其他特殊字符
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    
    # 将过滤后的文本转换为小写
    cleaned_text = cleaned_text.lower()
    
    return cleaned_text

def slide_word(text: str, l: int = 5) -> list:
    """
    滑动取词器
    Input: text='abcd',l=2
    Output: ['ab','bc','cd']
    :param text: 过滤后的文本 （只包含小写数字/字母）
    :param l: 滑动窗口长度，默认为 5
    :return:
    """
    # 使用正则表达式去除标点符号和其他特殊字符
    tf = re.sub(r'[^\w\s]', '', text)
    
    # 使用切片操作生成滑动窗口
    result = [tf[i:i + l] for i in range(len(tf) - l + 1)]
    
    return result

'''

In [ ]:
# The train set
text_file = open("words_250000_train.txt","r")
full_dictionary = text_file.read().splitlines()
for i in range(len(full_dictionary)):
    full_dictionary[i] = '@' + full_dictionary[i] + '#'

#select out the words in train set that has the same length with masked word.(take len = 5 as example)
dictionary_len5 = [ii for ii in full_dictionary if len(ii) == (5+2) ]
dictionary_len5_string = "".join(dictionary_len5)

In [ ]:
#given a masked word
word = '@.....#';
used_letter_list = [];
all_letter_list ='a b c d e f g h i j k l m n o p q r s t u v w x y z'.split(' ')

In [ ]:
#function that calculates unigram prob:
def unigram_prob(word_list_string, used_letter_list):
    
    deleted_string = re.sub('[@#]','',word_list_string) #delete @ and #
    
    for used_letter in used_letter_list:
        #delete the letters which are used 
        deleted_string = re.sub(used_letter,'',deleted_string)
        
    deleted_string_len = len(deleted_string)
    uniproblist = defaultdict(float);
    count_letter = Counter(deleted_string).most_common();
    
    for letter , number in count_letter :
        uniproblist[letter] = number/deleted_string_len
        
    return uniproblist

In [ ]:
unigram_prob_list = unigram_prob(dictionary_len5_string,used_letter_list)
print(unigram_prob_list)

In [ ]:
#suppose it's correct:
word = '@a....#'
used_letter_list = ['a']; #update used letter list
avalible_letter_list = list(set(all_letter_list)- set(used_letter_list))

In [ ]:
def slide_word_with_pos(text: str, l: int = 5) -> list:
    
    text_len = len(text)
    # 使用切片操作生成滑动窗口
    result = [(tuple(range(i,i + l)), text[i:i + l] ) for i in range(text_len - l + 1)]
    
    return result

In [ ]:
print(full_dictionary[5000])
slide_word_with_pos(full_dictionary[5000],2)

In [ ]:
#calculate the bigram_dictionary once for all
full_dictionary_bigram=[]

for words in full_dictionary:
    full_dictionary_bigram.append(slide_word_with_pos(words,2))
    
full_dictionary_bigram = [item for sublist in full_dictionary_bigram for item in sublist]

bigram_count = Counter(full_dictionary_bigram).most_common()

In [ ]:
#calculate the bigram_dictionary once for all
full_dictionary_trigram=[]

for words in full_dictionary:
    full_dictionary_trigram.append(slide_word_with_pos(words,3))
    
full_dictionary_trigram = [item for sublist in full_dictionary_trigram for item in sublist]

trigram_count = Counter(full_dictionary_trigram).most_common()

In [ ]:
#calculate the bigram_dictionary once for all
full_dictionary_fourgram=[]

for words in full_dictionary:
    full_dictionary_fourgram.append(slide_word_with_pos(words,4))
    
full_dictionary_fourgram = [item for sublist in full_dictionary_fourgram for item in sublist]

fourgram_count = Counter(full_dictionary_fourgram).most_common()

In [ ]:
fourgram_count

In [ ]:
def bigramprob_with_pos(bigram_pattern_with_pos, bigram_count):
    '''
    given a bigram_pattern_with_pos '((1,2), 'a.')',
    output the possibility for each letter in the masked letter 
    '''
    pos = bigram_pattern_with_pos[0] #position tuple
    bigram_pattern = bigram_pattern_with_pos[1] #pattern
    
    knownletter = re.sub('\.','',bigram_pattern)
    
    
    bigram_pattern_matched =list(filter(lambda x: (re.match(bigram_pattern,x[0][1])) and \
                                        (not (re.match(knownletter+knownletter,x[0][1]))) and \
                                        (pos == x[0][0]), bigram_count))
    
    bigramproblist = defaultdict(float);
    total_len = sum([ii[1] for ii in bigram_pattern_matched]);
    
    for bigram , number in bigram_pattern_matched :
        bigramproblist[re.sub(knownletter,'',bigram[1])] = number/total_len
        
    return bigramproblist

In [ ]:
def trigramprob_with_pos(trigram_pattern_with_pos, trigram_count):
    '''
    given a trigram_pattern_with_pos '((1,2,3), 'ab.')',
    output the possibility for each letter in the masked letter 
    '''
    pos = trigram_pattern_with_pos[0] #position tuple
    trigram_pattern = trigram_pattern_with_pos[1] #pattern
    
    knownletter = set(re.sub('\.','',trigram_pattern))
    
    trigram_pattern_matched = list( filter(lambda x: (pos == x[0][0]) and \
                                           (re.match(trigram_pattern,x[0][1])) and \
                                           (not set(x[0][1]).issubset(knownletter) ) , trigram_count) )
    trigramproblist = defaultdict(float);
    total_len = sum([ii[1] for ii in trigram_pattern_matched]);
    
    for trigram , number in trigram_pattern_matched :
        trigramproblist[re.sub(str(list(knownletter)) , '' ,trigram[1]) ] = number/total_len
    
    return trigramproblist

In [ ]:
def fourgramprob_with_pos(four_gram_pattern_with_pos, four_gram_count):
    '''
    given a fourgram_pattern_with_pos '((1,2,3,4), 'abc.')',
    output the possibility for each letter in the masked letter 
    '''
    pos = four_gram_pattern_with_pos[0] #position tuple
    fourgram_pattern = four_gram_pattern_with_pos[1] #pattern
    
    knownletter = set(re.sub('\.','',fourgram_pattern))
    
    fourgram_pattern_matched = list( filter(lambda x: (pos == x[0][0]) and \
                                           (re.match(fourgram_pattern,x[0][1])) and \
                                           (not set(x[0][1]).issubset(knownletter) ) , fourgram_count) )
    
    fourgramproblist = defaultdict(float);
    total_len = sum([ii[1] for ii in fourgram_pattern_matched]);
    
    for fourgram , number in fourgram_pattern_matched :
        fourgramproblist[re.sub(str(list(knownletter)) , '' ,fourgram[1]) ] = number/total_len
    return fourgramproblist

In [ ]:
fourgramprob_with_pos( ((2,3,4,5),'ab.a') , fourgram_count)

In [ ]:
trigramprob_with_pos( ((1,2,3),'ab.'), trigram_count)

In [ ]:
bigramprob_with_pos(((1,2),'a.'),bigram_count)

In [ ]:
def possible_pattern_exact_pos(word,pos,n):
    '''
    give a masked word , and output all possible n-gram pattern for a exact masked letter 
    '''
    shift = n-1
    word_last_pos = len(word) - 1; 
    start = pos - shift;
    start_list = list(range(start,start+n))
    slice_list = [list(range(i,i+n)) for i in start_list ]
    slice_list = list(filter((lambda x: x[0]>=0 and x[-1]<=word_last_pos) ,slice_list));
    
    pattern_list = []
    for poslist in slice_list:
        pattern = '';
        for pos in poslist:
            pattern += word[pos]
        
        pattern_list.append((tuple(poslist),pattern))
        
    pattern_list = list(filter(lambda x: x[1].count('.') == 1 ,pattern_list))
    return pattern_list

#for a given masked word, we anaylse each masked letter's possion n-gram 


def possible_pattern(word):
    mask_id = [];
    for i in range(len(word)):
        if word[i] == '.':
            mask_id.append(i)
            
    pattern_list_all = [[] for i in mask_id];
    
    for inx , i in enumerate(mask_id):
        for n in range(1,5):
            pattern_list = possible_pattern_exact_pos(word,i,n)
            if pattern_list != []:
                pattern_list_all[inx] += pattern_list
                
        #pattern_list_all[inx] = [(i,j) for j in pattern_list_all[inx]]
            
    return pattern_list_all

In [ ]:
possible_pattern_exact_pos('@a.b.aus.#',2,4)

In [ ]:
possible_pattern('@a.b.aus.#')

In [ ]:
word = '@abc..#'
used_letter_list = ['a','b','c'];
#update the unigram prob list
unigram_prob_list = unigram_prob(dictionary_len5_string,used_letter_list)
print(unigram_prob_list)

In [ ]:
allpattern = possible_pattern(word)
print(allpattern)

In [ ]:
#define a fucntion that can calculate the posibility for fix pos
def prob_fix_pos(pattern_list:list, unigram_prob_list, bigram_count ,trigram_count,fourgram_count):
    
    unigram_case = list(filter(lambda x: len(x[1])==1,pattern_list))
    bigram_case = list(filter(lambda x: len(x[1])==2,pattern_list)) 
    trigram_case = list(filter(lambda x: len(x[1])==3,pattern_list))
    fourgram_case = list(filter(lambda x: len(x[1])==4,pattern_list))
    
    bigram_prob_list = []
    trigram_prob_list = []
    fourgram_prob_list = []
    
    for pattern in bigram_case:
        bigram_prob_list.append( bigramprob_with_pos(pattern,bigram_count))
        
    for pattern in trigram_case:
        trigram_prob_list.append( trigramprob_with_pos(pattern,trigram_count))
        
    for pattern in fourgram_case:
        fourgram_prob_list.append( fourgramprob_with_pos(pattern,trigram_count))
    
    ngram_prob_list = defaultdict(float);
    bigram_prob_list_ave = defaultdict(float);
    trigram_prob_list_ave = defaultdict(float);
    fourgram_prob_list_ave = defaultdict(float);
    
    if len(bigram_prob_list) == 1:
        bigram_prob_list_ave = bigram_prob_list[0]
        
    elif len(bigram_prob_list) > 1:
        for key in bigram_prob_list[0]:
            num = 0;
            for i in range(len(bigram_prob_list)):
                 num += bigram_prob_list[i][key]
                
            bigram_prob_list_ave[key] = num/len(bigram_prob_list)
            
    if len(trigram_prob_list) == 1:
        trigram_prob_list_ave = trigram_prob_list[0]
        
    elif len(trigram_prob_list) > 1:
        for key in trigram_prob_list[0]:
            num = 0;
            for i in range(len(trigram_prob_list)):
                 num += trigram_prob_list[i][key]
                
            trigram_prob_list_ave[key] = num/len(trigram_prob_list)
        
    if len(fourgram_prob_list) == 1:
        fourgram_prob_list_ave = fourgram_prob_list[0]
        
    elif len(fourgram_prob_list) > 1:
        for key in fourgram_prob_list[0]:
            num = 0;
            for i in range(len(fourgram_prob_list)):
                 num += fourgram_prob_list[i][key]
                
            fourgram_prob_list_ave[key] = num/len(fourgram_prob_list)
                
    
    if bigram_prob_list_ave == {}:
        ngram_prob_list = unigram_prob_list
        
    elif ((bigram_prob_list_ave) != {}) and ((trigram_prob_list_ave) == {}):
        for key in unigram_prob_list.keys():
            ngram_prob_list[key] = 0.3*unigram_prob_list[key] + 0.7*bigram_prob_list_ave[key]
        
    elif ((bigram_prob_list_ave) != {}) and ((trigram_prob_list_ave) != {}):
        for key in unigram_prob_list.keys():
            ngram_prob_list[key] = 0.1*unigram_prob_list[key] + 0.3*bigram_prob_list_ave[key] + \
                                                                0.6*trigram_prob_list_ave[key]
            
    elif ((fourgram_prob_list_ave) != {}):
        for key in unigram_prob_list.keys():
            ngram_prob_list[key] = 0.05*unigram_prob_list[key] + 0.15*bigram_prob_list_ave[key] + \
                                   0.3*trigram_prob_list_ave[key] + 0.5*fourgram_prob_list_ave[key]
    
    return ngram_prob_list


def guess_letter(allpattern,unigram_prob_list,bigram_count, trigram_count ,fourgram_count):
    letter_prob_dic_list = [];
    
    for i in allpattern:
        letter_prob_dic_list.append(prob_fix_pos(i,unigram_prob_list,bigram_count,trigram_count,
                                                 fourgram_count))
        
    letter_prob_list = defaultdict(float);
    
    for key in letter_prob_dic_list[0].keys():
        for i in range(len(letter_prob_dic_list)):
            letter_prob_list[key] += letter_prob_dic_list[i][key]
            
    letter_prob_list = sorted(letter_prob_list.items(), key = lambda item: item[1],reverse=True)
    
    return letter_prob_list[0][0]

In [ ]:
allpattern[0]

In [ ]:
prob_fix_pos(allpattern[0], unigram_prob_list, bigram_count, trigram_count , fourgram_count)

In [ ]:
#return the most possible letter
guessedletter = guess_letter(allpattern, unigram_prob_list ,bigram_count,trigram_count,fourgram_count)
print(guessedletter)